About Dataset

Please note that this is the original dataset with additional information and proper attribution. There is at least one other version of this dataset on Kaggle that was uploaded without permission. Please be fair and attribute the original author.
This synthetic dataset is modeled after an existing milling machine and consists of 10 000 data points from a stored as rows with 14 features in columns

1. UID: unique identifier ranging from 1 to 10000

2. product ID: consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number

3. type: just the product type L, M or H from column 2

4. air temperature [K]: generated using a random walk process later normalized to a standard deviation of 2 K around 300 K

5. process temperature [K]: generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.

6. rotational speed [rpm]: calculated from a power of 2860 W, overlaid with a normally distributed noise

7. torque [Nm]: torque values are normally distributed around 40 Nm with a SD = 10 Nm and no negative values.

8. tool wear [min]: The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process.

9. a 'machine failure' label that indicates, whether the machine has failed in this particular datapoint for any of the following failure modes are true.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Layer, Softmax
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
df = pd.read_csv("C:/Users/Dorosh/Project/Predictive Maintenance Dataset (AI4I 2020)/ai4i2020.csv")
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [3]:
df.drop(columns=['UDI', 'Product ID'], inplace=True)
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     10000 non-null  object 
 1   Air temperature [K]      10000 non-null  float64
 2   Process temperature [K]  10000 non-null  float64
 3   Rotational speed [rpm]   10000 non-null  int64  
 4   Torque [Nm]              10000 non-null  float64
 5   Tool wear [min]          10000 non-null  int64  
 6   Machine failure          10000 non-null  int64  
 7   TWF                      10000 non-null  int64  
 8   HDF                      10000 non-null  int64  
 9   PWF                      10000 non-null  int64  
 10  OSF                      10000 non-null  int64  
 11  RNF                      10000 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 937.6+ KB


In [5]:
df = pd.get_dummies(df, columns=["Type"])

In [6]:
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,False,False,True
1,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,False,True,False
2,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,False,True,False
3,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,False,True,False
4,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,False,True,False


In [7]:
scaler = StandardScaler()
scaled_values = scaler.fit_transform(df[["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]"]])
scaled_df = pd.DataFrame(scaled_values, columns=["Air temperature", "Process temperature", "Rotational speed", "Torque"])
df[["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]"]] = scaled_values

In [8]:
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,-0.952389,-0.947360,0.068185,0.282200,0,0,0,0,0,0,0,False,False,True
1,-0.902393,-0.879959,-0.729472,0.633308,3,0,0,0,0,0,0,False,True,False
2,-0.952389,-1.014761,-0.227450,0.944290,5,0,0,0,0,0,0,False,True,False
3,-0.902393,-0.947360,-0.590021,-0.048845,7,0,0,0,0,0,0,False,True,False
4,-0.902393,-0.879959,-0.729472,0.001313,9,0,0,0,0,0,0,False,True,False


In [9]:
df["Machine failure"].unique()

array([0, 1])

In [10]:
X = df.drop(columns=["Machine failure"])
y = df["Machine failure"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
class CustomDenseLayer(Layer):
    def __init__(self, units = 128):
        super(CustomDenseLayer, self).__init__()
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight(shape = (input_shape[-1], self.units),
                                 initializer = 'random_normal',
                                 trainable = True)
        self.b = self.add_weight(shape = (self.units,),
                                 initializer = 'zeros',
                                 trainable = True)
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

In [14]:
model = Sequential([
    CustomDenseLayer(128),
    Dropout(0.2),
    CustomDenseLayer(64),
    Dropout(0.2),
    CustomDenseLayer(32),
    Softmax()])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=16, batch_size=32)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

Epoch 1/16

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.9020 - loss: 1.0227
Epoch 2/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.9683 - loss: 0.2279
Epoch 3/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.9664 - loss: 0.1686
Epoch 4/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - accuracy: 0.9795 - loss: 0.0708
Epoch 5/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.9845 - loss: 0.0600
Epoch 6/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.9916 - loss: 0.0465
Epoch 7/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.9971 - loss: 0.0156
Epoch 8/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.9989 - loss: 0.0116
Epoch 9/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.9973 - loss: 0.0193
Epoch 10/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.9967 - loss: 0.0140
Epoch 11/16
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.9988 - loss: 0.0126
Epoch 12/16
250/250 ━━━━━━━━━